In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import r2_score


In [3]:
import torch

print("Is CUDA available: ", torch.cuda.is_available())
print("CUDA version: ", torch.version.cuda)
print("Number of CUDA devices: ", torch.cuda.device_count())
print("CUDA Device Name: ", torch.cuda.get_device_name(0) if torch.cuda.device_count() > 0 else "No CUDA device")


Is CUDA available:  False
CUDA version:  None
Number of CUDA devices:  0
CUDA Device Name:  No CUDA device


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

cpu


In [ ]:
row_data_file = 'TSLA_stock_data_2023.csv'
predict_data_file = 'TSLA_stock_data_2024.csv'

In [5]:
# Creating sliding windows
def slide_windows(features):

    # We need to ensure we have data for t-1, t, t+1 without index errors
    windowed_data = []
    predict_prices = []  # List to store the target 'Close' prices
    cycle_data=[]

    # Handling daily cycles
    cycle_length = 7
    for cycle in range(len(features) - cycle_length + 1):
        start_index = cycle
        end_index = start_index + cycle_length
        cycle_datas = features.iloc[start_index:end_index,1:6].values
        cycle_data.append(cycle_datas)
        
    # Creating sliding windows within the cycle
    for i in range(14,len(cycle_data)-7):  # Avoiding index error by stopping before the last day
        pre_previous = cycle_data[i - 14]
        previous = cycle_data[i- 7]
        current_state = cycle_data[i]

        combined_features = np.concatenate([pre_previous, previous, current_state]).reshape(1,-1).squeeze()
        windowed_data.append(combined_features)
        predict_prices.append(cycle_data[i+7][-1][3])
    
    return windowed_data, predict_prices


In [6]:
# Load data
row_data = pd.read_csv(row_data_file)

windowed_data, predict_prices = slide_windows(row_data)
# Convert to PyTorch tensors
state = torch.tensor(windowed_data, dtype=torch.float32).to(device)
predict_price = torch.tensor(predict_prices, dtype=torch.float32).to(device)
# state[2]

C:\Users\26210\AppData\Local\Temp\ipykernel_22108\181566533.py:6: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:278.)
  state = torch.tensor(windowed_data, dtype=torch.float32).to(device)


In [7]:
# divide the data into training part and test part
state_train, state_test, predict_price_train,predict_price_test= train_test_split(state, predict_price,test_size=0.2, random_state=42)
train_loader = DataLoader(TensorDataset(state_train, predict_price_train), batch_size=200, shuffle=True)
test_loader = DataLoader(TensorDataset(state_test, predict_price_test), batch_size=300)


In [8]:
# construct NN
class NN(nn.Module):
    def __init__(self, n_observations):
        super(NN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, 1)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)
    

In [9]:
# Optimazation function, it do one step of gredient decent, 
def optimize_model():
    for state, target in train_loader:
        current_value=value_net(state).squeeze()              
        criterion = nn.SmoothL1Loss()
        loss=criterion(current_value,target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    for state, target in test_loader:
        with torch.no_grad():
            test_valur=value_net(state).squeeze()                 
            l_test=criterion(test_valur,target)
    return loss.item(),l_test.item()

In [10]:
learning_rates = [1e-5, 1e-3]  # Possible learning rates
weight_decays = [1e-2, 1e-3, 1e-4, 1e-5]  # Possible weight decay values
n_observations = len(state[0])
epochs=2000

best_lr = None
best_weight_decay = None
lowest_test_loss = float('inf')  # Initialize with infinity to ensure any first result is better

# value_net is your prediction function, take state as input and output is the prediction price
value_net = NN(n_observations).to(device)

for LR in learning_rates:
    for wd in weight_decays:
        print(f"Testing with LR = {LR} and Weight Decay = {wd}")
        # 'optimize'  is an easy package to do Gredient decent, 'Adam' is a method to let learing rate decay as step go.
        optimizer = optim.Adam(value_net.parameters(), lr=LR,weight_decay=wd)
    
        # Run the training loop for this combination of parameters
        for epoch in range(epochs):
            l_train, l_test = optimize_model()  # Use the current lr and wd in your optimization
            if epoch % 100 == 0:  # Report every 100 epochs
                print(f'Epoch [{epoch+1}/{epochs}], L_train: {l_train}, L_test: {l_test}')

            # Save parameters if this is the best we've seen
            if l_test < lowest_test_loss:
                lowest_test_loss = l_test
                best_lr = LR
                best_weight_decay = wd

# Output the best parameters and the test loss achieved with them
print(f"Best Learning Rate: {best_lr}, Best Weight Decay: {best_weight_decay}, Lowest L_test: {lowest_test_loss}")
#

Testing with LR = 1e-10 and Weight Decay = 0.01
Epoch [1/2000], L_train: 404807.625, L_test: 399929.78125
Epoch [101/2000], L_train: 349372.5, L_test: 399929.21875
Epoch [201/2000], L_train: 333433.46875, L_test: 399928.59375
Epoch [301/2000], L_train: 367545.4375, L_test: 399928.0
Epoch [401/2000], L_train: 365227.375, L_test: 399927.4375
Epoch [501/2000], L_train: 388198.03125, L_test: 399926.875
Epoch [601/2000], L_train: 342717.65625, L_test: 399926.3125
Epoch [701/2000], L_train: 341645.78125, L_test: 399925.78125
Epoch [801/2000], L_train: 363376.3125, L_test: 399925.15625
Epoch [901/2000], L_train: 387314.9375, L_test: 399924.5625
Epoch [1001/2000], L_train: 405618.21875, L_test: 399924.03125
Epoch [1101/2000], L_train: 373733.4375, L_test: 399923.5
Epoch [1201/2000], L_train: 357462.46875, L_test: 399922.84375
Epoch [1301/2000], L_train: 351467.5, L_test: 399922.3125
Epoch [1401/2000], L_train: 391673.8125, L_test: 399921.6875
Epoch [1501/2000], L_train: 351041.96875, L_test: 3

In [11]:
# continue optimize
# start from last training result
epochs=10000
LR = 1e-15
Weight_decay=0.001
value_net.train()

run_count = 0 

while True:
    for epoch in range(epochs):
        l_train, l_test = optimize_model()
        if epoch % 10 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], L_train: {l_train}, L_test: {l_test}')
    
    run_count += 1

    # Check the stopping condition
    if l_test <= 10 or run_count > 100:
        print("Stopping the loop.")
        break


Epoch [1/100000], L_train: 59.46142578125,L_test: 66.21602630615234
Epoch [11/100000], L_train: 59.95954132080078,L_test: 72.85272216796875
Epoch [21/100000], L_train: 52.18948745727539,L_test: 72.85000610351562
Epoch [31/100000], L_train: 56.47296905517578,L_test: 66.0677719116211
Epoch [41/100000], L_train: 50.293174743652344,L_test: 65.43447875976562
Epoch [51/100000], L_train: 67.69593811035156,L_test: 66.10823822021484
Epoch [61/100000], L_train: 59.5383415222168,L_test: 62.41083908081055
Epoch [71/100000], L_train: 56.66339111328125,L_test: 63.117000579833984
Epoch [81/100000], L_train: 57.51463317871094,L_test: 65.15682983398438
Epoch [91/100000], L_train: 57.65017318725586,L_test: 62.998138427734375
Epoch [101/100000], L_train: 55.419715881347656,L_test: 64.20887756347656
Epoch [111/100000], L_train: 60.46889877319336,L_test: 67.38843536376953
Epoch [121/100000], L_train: 57.39326477050781,L_test: 62.74835968017578
Epoch [131/100000], L_train: 52.41047286987305,L_test: 62.82205

In [1]:
#save last line results
torch.save(value_net.state_dict(),'model_parameters.pth')

NameError: name 'torch' is not defined

In [ ]:
#load results
loaded_model = NN(n_observations).to(device)
value_net.load_state_dict(torch.load('model_parameters.pth'))

In [ ]:
# Load data to predict
windowed_data, predict_prices = slide_windows(pd.read_csv(predict_data_file))
# Convert to PyTorch tensors
state_new = torch.tensor(windowed_data, dtype=torch.float32).to(device)
real_price = torch.tensor(predict_prices, dtype=torch.float32).to(device)

#load the trained parameter to the model
# loaded_model = NN(n_observations).to(device)
# loaded_model.load_state_dict(torch.load('model_parameters.pth')) 
# value_net=loaded_model 

# predict price
value_net.eval()  
with torch.no_grad():
    criterion = nn.SmoothL1Loss()
    predict_price=loaded_model(state_new).squeeze() #predict price
    l_test=criterion(real_price,predict_price) #calculate difference
print(predict_price)
print(l_test)



In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(real_price, color='black', label='Real Stock Price')
plt.plot(predict_price, color='green', label='Predicted Stock Price in NN')
plt.title('Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(l_test, color='red', label='Prediction Error (%)')
plt.title('Prediction Error')
plt.xlabel('Time')
plt.ylabel('Error (%)')
plt.legend()
plt.show()

In [ ]:
r2 = r2_score(real_price, predict_prices)

print(f"Average NN Prediction Error (%): {np.mean(l_test)}")
print(f"Variance of NN Prediction Error (%): {np.var(l_test)}")
print(f"R^2 Score: {r2}")